In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.set_printoptions(threshold=200)
pd.set_option('display.max_columns', 200)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data/"))

# Any results you write to the current directory are saved as output.

['device_data.csv', 'pageviews_complemento.csv', 'sampleSubmission.csv', 'SITE_ID.csv', 'PAGE.csv', 'README.md', 'CONTENT_CATEGORY.csv', 'CONTENT_CATEGORY_BOTTOM.csv', 'pageviews.csv', 'conversiones.csv', 'CONTENT_CATEGORY_TOP.csv']


In [2]:
data = pd.concat([
       pd.read_csv("../data/pageviews.csv", parse_dates=["FEC_EVENT"]),
       pd.read_csv("../data/pageviews_complemento.csv", parse_dates=["FEC_EVENT"])
])

/home/horacio/.conda/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
data.head()

,CONTENT_CATEGORY,CONTENT_CATEGORY_BOTTOM,CONTENT_CATEGORY_TOP,FEC_EVENT,ON_SITE_SEARCH_TERM,PAGE,SITE_ID,USER_ID
0,1,1,1,2018-03-30 07:35:48,1,1,1,0
1,2,2,2,2018-03-30 07:35:52,1,2,2,0
2,2,2,2,2018-03-30 07:36:11,1,3,3,0
3,2,2,2,2018-03-30 07:36:16,1,4,3,0
4,2,2,2,2018-03-30 07:41:38,1,5,2,0


### Benchmark Kernel

In [4]:
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [5]:
X_test.head(5)

,CONTENT_CATEGORY_1,CONTENT_CATEGORY_2,CONTENT_CATEGORY_3,CONTENT_CATEGORY_4,CONTENT_CATEGORY_5,CONTENT_CATEGORY_6,CONTENT_CATEGORY_7,CONTENT_CATEGORY_8,CONTENT_CATEGORY_9,CONTENT_CATEGORY_10,CONTENT_CATEGORY_11,CONTENT_CATEGORY_12,CONTENT_CATEGORY_13,CONTENT_CATEGORY_14,CONTENT_CATEGORY_15,CONTENT_CATEGORY_16,CONTENT_CATEGORY_17,CONTENT_CATEGORY_18,CONTENT_CATEGORY_19,CONTENT_CATEGORY_20,CONTENT_CATEGORY_21,CONTENT_CATEGORY_22,CONTENT_CATEGORY_23,CONTENT_CATEGORY_24,CONTENT_CATEGORY_25,CONTENT_CATEGORY_26,CONTENT_CATEGORY_27,CONTENT_CATEGORY_28,CONTENT_CATEGORY_29,CONTENT_CATEGORY_30,CONTENT_CATEGORY_31,CONTENT_CATEGORY_32,CONTENT_CATEGORY_33,CONTENT_CATEGORY_34,CONTENT_CATEGORY_35,CONTENT_CATEGORY_36,CONTENT_CATEGORY_37,CONTENT_CATEGORY_38,CONTENT_CATEGORY_39,CONTENT_CATEGORY_40,CONTENT_CATEGORY_41,CONTENT_CATEGORY_42,CONTENT_CATEGORY_43,CONTENT_CATEGORY_44,CONTENT_CATEGORY_45,CONTENT_CATEGORY_46,CONTENT_CATEGORY_47,CONTENT_CATEGORY_48,CONTENT_CATEGORY_49,CONTENT_CATEGORY_50,CONTENT_CATEGORY_51,CONTENT_CATEGORY_52,CONTENT_CATEGORY_53,CONTENT_CATEGORY_54,CONTENT_CATEGORY_55,CONTENT_CATEGORY_56,CONTENT_CATEGORY_57,CONTENT_CATEGORY_58,CONTENT_CATEGORY_59,CONTENT_CATEGORY_60,CONTENT_CATEGORY_61,CONTENT_CATEGORY_62,CONTENT_CATEGORY_63,CONTENT_CATEGORY_64,CONTENT_CATEGORY_68,CONTENT_CATEGORY_BOTTOM_1,CONTENT_CATEGORY_BOTTOM_2,CONTENT_CATEGORY_BOTTOM_3,CONTENT_CATEGORY_BOTTOM_4,CONTENT_CATEGORY_BOTTOM_5,CONTENT_CATEGORY_BOTTOM_6,CONTENT_CATEGORY_BOTTOM_7,CONTENT_CATEGORY_BOTTOM_8,CONTENT_CATEGORY_BOTTOM_9,CONTENT_CATEGORY_BOTTOM_10,CONTENT_CATEGORY_BOTTOM_11,CONTENT_CATEGORY_BOTTOM_12,CONTENT_CATEGORY_BOTTOM_13,CONTENT_CATEGORY_BOTTOM_14,CONTENT_CATEGORY_BOTTOM_15,CONTENT_CATEGORY_BOTTOM_16,CONTENT_CATEGORY_BOTTOM_17,CONTENT_CATEGORY_BOTTOM_18,CONTENT_CATEGORY_BOTTOM_19,CONTENT_CATEGORY_BOTTOM_20,CONTENT_CATEGORY_BOTTOM_21,CONTENT_CATEGORY_BOTTOM_22,CONTENT_CATEGORY_BOTTOM_23,CONTENT_CATEGORY_BOTTOM_24,CONTENT_CATEGORY_BOTTOM_25,CONTENT_CATEGORY_BOTTOM_26,CONTENT_CATEGORY_BOTTOM_27,CONTENT_CATEGORY_BOTTOM_28,CONTENT_CATEGORY_BOTTOM_29,CONTENT_CATEGORY_BOTTOM_30,CONTENT_CATEGORY_BOTTOM_31,CONTENT_CATEGORY_BOTTOM_32,CONTENT_CATEGORY_BOTTOM_33,CONTENT_CATEGORY_BOTTOM_34,CONTENT_CATEGORY_BOTTOM_35,...,PAGE_1740,PAGE_1741,PAGE_1742,PAGE_1743,PAGE_1744,PAGE_1745,PAGE_1746,PAGE_1747,PAGE_1748,PAGE_1749,PAGE_1750,PAGE_1751,PAGE_1752,PAGE_1753,PAGE_1754,PAGE_1755,PAGE_1756,PAGE_1757,PAGE_1758,PAGE_1759,PAGE_1760,PAGE_1761,PAGE_1762,PAGE_1763,PAGE_1764,PAGE_1765,PAGE_1766,PAGE_1767,PAGE_1768,PAGE_1769,PAGE_1770,PAGE_1771,PAGE_1772,PAGE_1773,PAGE_1774,PAGE_1775,PAGE_1776,PAGE_1777,PAGE_1778,PAGE_1779,PAGE_1780,PAGE_1781,PAGE_1782,PAGE_1783,PAGE_1784,PAGE_1785,PAGE_1786,PAGE_1787,PAGE_1788,PAGE_1789,PAGE_1790,PAGE_1791,PAGE_1792,PAGE_1793,PAGE_1794,PAGE_1795,PAGE_1796,PAGE_1797,PAGE_1798,PAGE_1799,PAGE_1800,PAGE_1801,PAGE_1802,PAGE_1803,PAGE_1804,PAGE_1805,PAGE_1806,PAGE_1807,PAGE_1808,PAGE_1809,PAGE_1810,PAGE_1811,PAGE_1812,PAGE_1813,PAGE_1814,PAGE_1815,PAGE_1816,PAGE_1817,PAGE_1818,PAGE_1819,PAGE_1820,PAGE_1821,PAGE_1822,PAGE_1823,PAGE_1824,PAGE_1825,PAGE_1826,PAGE_1827,PAGE_1828,PAGE_1829,PAGE_1830,PAGE_1831,PAGE_1832,PAGE_1833,PAGE_1834,PAGE_1835,SITE_ID_1,SITE_ID_2,SITE_ID_3,SITE_ID_4
USER_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.028767,0.478153,0.000155,0.486705,0.003732,0.001399,0.000466,0.000155,0.000155,0.000155,0.000000,0.000000,0.000155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.028767,0.478153,0.000155,0.486705,0.003732,0.001399,0.000466,0.000155,0.000155,0.000155,0.000000,0.000000,0.000155,0.000000,0.000000,0.000000,0.000000,0.000000,

In [6]:
X_test.shape

(11676, 2174)

In [7]:
X_test.iloc[0,0:65].sum() # la suma de las frecuencias debe ser 1

1.0

In [8]:
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [240]:
X_train.iloc[:,56]

USER_ID
0        0.000204
1        0.000000
2        0.000000
3        0.001287
4        0.006457
5        0.000941
6        0.000000
7        0.000000
8        0.000000
9        0.000000
10       0.000000
11       0.001517
12       0.001445
13       0.002326
14       0.000000
15       0.000000
16       0.000000
17       0.000611
18       0.000000
19       0.000000
20       0.000000
21       0.000000
22       0.007153
23       0.001239
24       0.000801
25       0.000000
26       0.000000
27       0.000000
28       0.003674
29       0.001205
           ...   
11634    0.008772
11635    0.000000
11637    0.000000
11638    0.000000
11639    0.000000
11642    0.000000
11643    0.007371
11644    0.089286
11645    0.034161
11646    0.000000
11647    0.000000
11649    0.106557
11650    0.000000
11651    0.000000
11652    0.000000
11654    0.012821
11657    0.014184
11658    0.125000
11659    0.000000
11661    0.000000
11663    0.066667
11666    0.000000
11667    0.000000
11668    0.125000
11

In [10]:
X_train.iloc[0,0:65].sum()

1.0

In [11]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [12]:
y_prev = pd.read_csv("../data/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(set(X_train.index))
y_train.loc[list(idx)] = 1

In [13]:
print(X_train.shape, X_test.shape)

(11529, 2026) (11676, 2026)


In [272]:
from sklearn import model_selection
from lightgbm import LGBMClassifier

fi = [] # Creamos un objeto vacío para guardar la importancia de las variables de los modelos entrenados
test_probs = [] # Creamos un objeto vacío para guardar las probabilidades estimadas
i = 0 
for train_idx, valid_idx in model_selection.KFold(n_splits=10, shuffle=True).split(X_train): # Iterams sobre 10 folds que creamos para entrenar y validar
    i += 1
    Xt = X_train.iloc[train_idx] # Definimos el set de entrenamiento a partir de los indices que coincidan con los 9 folds de entrenamiento
    yt = y_train.loc[X_train.index].iloc[train_idx] # Definimos el Target de entrenamiento con los Usuarios que coincidan con los que se encuentran en estos folds

    Xv = X_train.iloc[valid_idx] # Definimos el set de validación a partir los indices que coincidan con el fold de validación
    yv = y_train.loc[X_train.index].iloc[valid_idx] # Definimos el Target de validación con los Usuarios que coincidan con los que se encuentran en este fold

    learner = LGBMClassifier(n_estimators=10000) # Entrenamos un modelo lightgbm con los parámetros por default usando un máximo de 10.000 árboles
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)]) # Definimos un early stop de 10 y el método de evaluación como AUC
    
    test_probs.append(pd.Series(learner.predict_proba(X_test)[:, -1],
                                index=X_test.index, name="fold_" + str(i))) # Predecimos sobre la base total y nos guardamos las probabilidades estimadas
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns)) # Guardamos la proporción de importancia de cada variable en el modelo

test_probs = pd.concat(test_probs, axis=1).mean(axis=1) # Caluclamos la media de las probabilidades
test_probs.index.name="USER_ID" # Renombramos las columnas
test_probs.name="SCORE" # Renombramos las columnas
test_probs.to_csv("respuesta", header=True, compression="zip") # Guardamos la predicción final en un zip para subirlo a la plataforma
fi = pd.concat(fi, axis=1).mean(axis=1) # Como explicación del modelo guardamos la importancia de las variables de los modelos entrenados

[1]	training's binary_logloss: 0.130823	training's auc: 0.841164	valid_1's binary_logloss: 0.129447	valid_1's auc: 0.778452
Training until validation scores don't improve for 10 rounds.
[2]	training's binary_logloss: 0.120115	training's auc: 0.886782	valid_1's binary_logloss: 0.126224	valid_1's auc: 0.825198
[3]	training's binary_logloss: 0.112091	training's auc: 0.914498	valid_1's binary_logloss: 0.124029	valid_1's auc: 0.818588
[4]	training's binary_logloss: 0.105792	training's auc: 0.922075	valid_1's binary_logloss: 0.122104	valid_1's auc: 0.821177
[5]	training's binary_logloss: 0.100675	training's auc: 0.927694	valid_1's binary_logloss: 0.120337	valid_1's auc: 0.823516
[6]	training's binary_logloss: 0.0957087	training's auc: 0.933953	valid_1's binary_logloss: 0.118139	valid_1's auc: 0.828445
[7]	training's binary_logloss: 0.0917389	training's auc: 0.941785	valid_1's binary_logloss: 0.11666	valid_1's auc: 0.839352
[8]	training's binary_logloss: 0.0881612	training's auc: 0.950103	val

[12]	training's binary_logloss: 0.075245	training's auc: 0.969246	valid_1's binary_logloss: 0.118368	valid_1's auc: 0.866474
[13]	training's binary_logloss: 0.0726635	training's auc: 0.971093	valid_1's binary_logloss: 0.118271	valid_1's auc: 0.863866
[14]	training's binary_logloss: 0.0703882	training's auc: 0.973333	valid_1's binary_logloss: 0.117511	valid_1's auc: 0.867501
[15]	training's binary_logloss: 0.0681373	training's auc: 0.976658	valid_1's binary_logloss: 0.117308	valid_1's auc: 0.865837
[16]	training's binary_logloss: 0.0662733	training's auc: 0.977767	valid_1's binary_logloss: 0.117225	valid_1's auc: 0.86494
[17]	training's binary_logloss: 0.0640762	training's auc: 0.984129	valid_1's binary_logloss: 0.116996	valid_1's auc: 0.863665
[18]	training's binary_logloss: 0.0622006	training's auc: 0.985222	valid_1's binary_logloss: 0.116373	valid_1's auc: 0.863382
[19]	training's binary_logloss: 0.0603362	training's auc: 0.989085	valid_1's binary_logloss: 0.116044	valid_1's auc: 0.8

[1]	training's binary_logloss: 0.124981	training's auc: 0.841543	valid_1's binary_logloss: 0.164553	valid_1's auc: 0.771173
Training until validation scores don't improve for 10 rounds.
[2]	training's binary_logloss: 0.115261	training's auc: 0.86716	valid_1's binary_logloss: 0.159394	valid_1's auc: 0.785313
[3]	training's binary_logloss: 0.107961	training's auc: 0.908502	valid_1's binary_logloss: 0.157849	valid_1's auc: 0.805317
[4]	training's binary_logloss: 0.101586	training's auc: 0.916418	valid_1's binary_logloss: 0.155133	valid_1's auc: 0.806184
[5]	training's binary_logloss: 0.0963318	training's auc: 0.923527	valid_1's binary_logloss: 0.152977	valid_1's auc: 0.812425
[6]	training's binary_logloss: 0.0917766	training's auc: 0.936522	valid_1's binary_logloss: 0.151367	valid_1's auc: 0.812491
[7]	training's binary_logloss: 0.0875033	training's auc: 0.941152	valid_1's binary_logloss: 0.150343	valid_1's auc: 0.815809
[8]	training's binary_logloss: 0.0839259	training's auc: 0.953831	va

[10]	training's binary_logloss: 0.0791918	training's auc: 0.959917	valid_1's binary_logloss: 0.119668	valid_1's auc: 0.757241
[11]	training's binary_logloss: 0.076187	training's auc: 0.96559	valid_1's binary_logloss: 0.119002	valid_1's auc: 0.77819
[12]	training's binary_logloss: 0.0734205	training's auc: 0.969734	valid_1's binary_logloss: 0.119034	valid_1's auc: 0.778255
[13]	training's binary_logloss: 0.0707791	training's auc: 0.975405	valid_1's binary_logloss: 0.119033	valid_1's auc: 0.77288
[14]	training's binary_logloss: 0.0683762	training's auc: 0.978759	valid_1's binary_logloss: 0.118072	valid_1's auc: 0.778742
[15]	training's binary_logloss: 0.0663501	training's auc: 0.979784	valid_1's binary_logloss: 0.118123	valid_1's auc: 0.776613
[16]	training's binary_logloss: 0.0643144	training's auc: 0.981058	valid_1's binary_logloss: 0.11775	valid_1's auc: 0.777624
[17]	training's binary_logloss: 0.0622123	training's auc: 0.983788	valid_1's binary_logloss: 0.118212	valid_1's auc: 0.7750

# Lado B

### Experimental models

#### Train set and Test set

We split the test set, so we use last three months to test.

In [14]:
data = data[data.FEC_EVENT.dt.month < 10]
X_test2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test2 = pd.concat(X_test2, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [15]:
data = data[(data.FEC_EVENT.dt.month < 7)]
X_train2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train2 = pd.concat(X_train2, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [16]:
y_prev2 = pd.read_csv("../data/conversiones.csv")
y_test2 = pd.Series(0, index=X_test2.index)
idx = set(y_prev2[y_prev2.mes >= 10].USER_ID.unique()).intersection(set(X_test2.index))
y_test2.loc[list(idx)] = 1

In [17]:
y_train2 = pd.Series(0, index=X_train2.index)
idx = set(y_prev2[(y_prev2.mes < 10) & (y_prev2.mes > 6)].USER_ID.unique()).intersection(set(X_train2.index))
y_train2.loc[list(idx)] = 1

In [18]:
print(X_train2.shape, y_train2.shape, X_test2.shape, y_test2.shape)

(11314, 1890) (11314,) (11529, 2026) (11529,)


In [19]:
features = list(set(X_train2.columns).intersection(set(X_test2.columns)))
X_train2 = X_train2[features]
X_test2 = X_test2[features]

In [20]:
X_train2.shape

(11314, 1890)

In [21]:
from sklearn.decomposition import PCA

pca2 = PCA(0.99,random_state=23)

In [22]:
pca2.fit(X_train2)

PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=23,
  svd_solver='auto', tol=0.0, whiten=False)

In [23]:
X_train_pca2 = pd.DataFrame(pca2.transform(X_train2))
X_test_pca2 = pd.DataFrame(pca2.transform(X_test2))

In [24]:
print(X_train_pca2.shape, X_test_pca2.shape)

(11314, 43) (11529, 43)


In [25]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selected_features = SelectKBest(chi2,k=200)
selected_features.fit(X_train2, y_train2)
X_train_skb2 = pd.DataFrame(selected_features.transform(X_train2))
X_test_skb2 = pd.DataFrame(selected_features.transform(X_test2))

In [198]:
p_values = pd.DataFrame(selected_features.pvalues_.round(5))
p_values.index.name="FEAT_N"
p_values.columns = ["P_VALUES"]
p_values.sort_values(by=['P_VALUES'])

,P_VALUES
FEAT_N,
56,0.00009
1108,0.00018
1829,0.00141
1568,0.00358
643,0.01056
161,0.01056
1664,0.05386
1607,0.07726
352,0.14028


In [199]:
df = pd.DataFrame(selected_features.scores_.round(5))
df.index.name="FEAT_N"
df.columns = ["SCORE"]
df.sort_values(by=['SCORE'],ascending=False)

,SCORE
FEAT_N,
56,15.29086
1108,14.03890
1829,10.18657
1568,8.48367
643,6.53739
161,6.53739
1664,3.71697
1607,3.12166
352,2.17490


In [190]:
X_train.columns[56]

'PAGE_41'

In [191]:
X_train.columns[1108]

'PAGE_110'

In [241]:
X_train_cut1 = X_train2.drop(columns=['PAGE_110'])
X_test_cut1 = X_test2.drop(columns=['PAGE_110'])

In [242]:
selected_features = SelectKBest(chi2,k=200)
selected_features.fit(X_train_cut1, y_train2)
X_train_cut1 = pd.DataFrame(selected_features.transform(X_train_cut1))
X_test_cut1 = pd.DataFrame(selected_features.transform(X_test_cut1))

In [212]:
X_train_cut1.shape

(11314, 200)

In [197]:
import sys
import numpy
numpy.set_printoptions(threshold=50)

In [27]:
selected_features = SelectKBest(chi2,k=200)
selected_features.fit(X_train, y_train)
X_train_skb_compl = pd.DataFrame(selected_features.transform(X_train))
X_test_skb_compl = pd.DataFrame(selected_features.transform(X_test))

### Decision Tree Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(random_state=47)

params = {
    'criterion' : ['gini','entropy'],
    'splitter' : ['best','random'],
    'max_depth' : [None,3,4,5,8,10],
    'max_features' : [None, 10, 25]
}

gs = GridSearchCV(dtc, params, scoring='roc_auc', cv=5)

In [ ]:
gs2 = gs.fit(X_train2,y_train2)

In [ ]:
gs2.best_score_

In [ ]:
gs2.best_estimator_

In [ ]:
opt_arbol1 = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=47,
            splitter='best')

In [ ]:
predictor2 = opt_arbol1.fit(X_train2,y_train2)

In [ ]:
prediccion2 = predictor2.predict_proba(X_test2)

In [24]:
from sklearn.metrics import roc_auc_score

scores = roc_auc_score(y_test2,prediccion2[:,1])
scores

NameError: name 'prediccion2' is not defined

In [ ]:
gs2_no_cv = GridSearchCV(dtc,params,scoring='roc_auc',cv=2)
gs2_no_cv.fit(X_train2,y_train2)
pred_no_cv = gs2_no_cv.best_estimator_
predijo = pred_no_cv.predict_proba(X_test2)

In [ ]:
scores_no_cv = roc_auc_score(y_test2,predijo[:,1])
scores_no_cv

In [ ]:
gs_pca = gs.fit(X_train_pca2,y_train2)

In [ ]:
gs_pca.best_score_

In [ ]:
predictor_pca = gs_pca.best_estimator_
pred_pca = predictor_pca.predict_proba(X_test_pca2)

In [ ]:
scores_pca = roc_auc_score(y_test2,pred_pca[:,1])
scores_pca

In [ ]:
gs_skb = gs.fit(X_train_skb2,y_train2)

In [ ]:
gs_skb.best_score_

In [ ]:
predictor_skb = gs_skb.best_estimator_
predictor_skb

In [ ]:
predic_skb = predictor_skb.predict_proba(X_test_skb2)

In [ ]:
scores_skb_tree = roc_auc_score(y_test2,predic_skb[:,1])
scores_skb_tree

#### Modelo Final

In [ ]:
arbol1 = predictor_skb.fit(X_train_skb_compl,y_train)
sol_arbol1 = arbol1.predict_proba(X_test_skb_compl)

In [ ]:
resultado_arbol1 = pd.Series(sol_arbol1[:,1])
resultado_arbol1.index.name="USER_ID" # Renombramos las columnas
resultado_arbol1.name="SCORE" # Renombramos las columnas
resultado_arbol1.to_csv("respuesta_arbol_1", header=True, compression="zip")

### Otro Arbol

In [ ]:
# Optimization through GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

pg = [
    {"splitter":["best","random"],
     "max_depth":[None,1,2,3,4,5,7,10], 
     "max_features":[10,25], 
     "max_leaf_nodes":[10,50,100,150,200]}
]

dtc2 = arbol1

opt = GridSearchCV(dtc2, pg, scoring="roc_auc",n_jobs=-1,cv=StratifiedKFold(5))

In [ ]:
opt.fit(X_train2, y_train2)

In [ ]:
opt.best_score_

In [ ]:
optimised_decision_tree = opt.best_estimator_
optimised_decision_tree

In [ ]:
sol = optimised_decision_tree.predict_proba(X_test2)

In [ ]:
scores = roc_auc_score(y_test2,sol[:,1])
scores

In [ ]:
opt_arbol2 = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=25, max_leaf_nodes=10, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=47, splitter='best')

In [ ]:
opt_pca = opt.fit(X_train_pca2,y_train2)

In [ ]:
opt_pca.best_score_

In [ ]:
pca_tree_2 = opt_pca.best_estimator_

In [ ]:
res_tree_2 = pca_tree_2.predict_proba(X_test_pca2)

In [ ]:
scores = roc_auc_score(y_test2,res_tree_2[:,1])
scores

In [ ]:
opt_skb = opt.fit(X_train_skb2,y_train2)

In [ ]:
opt_skb.best_score_

In [ ]:
skb_tree_2 = opt_skb.best_estimator_

In [ ]:
sol_skb_2 = skb_tree_2.predict_proba(X_test_skb2)

In [ ]:
scores = roc_auc_score(y_test2,sol_skb_2[:,1])
scores

#### Archivo Final

In [ ]:
arbol2 = skb_tree_2
skb_tree_2.fit(X_train_skb_compl,y_train)

In [ ]:
sol_a2 = skb_tree_2.predict_proba(X_test_skb_compl)
resultado_arbol2 = pd.Series(sol_a2[:,1])
resultado_arbol2.index.name="USER_ID" # Renombramos las columnas
resultado_arbol2.name="SCORE" # Renombramos las columnas
resultado_arbol2.to_csv("respuesta_arbol_2", header=True, compression="zip")

### ADABOOST

In [151]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators = 8,random_state=23)
clf.fit(X_train2,y_train2)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=8, random_state=23)

In [152]:
probab = clf.predict_proba(X_test2)

In [153]:
scores = roc_auc_score(y_test2,probab[:,1])
scores

0.8264840335436348

In [154]:
clf_pca = clf.fit(X_train_pca2,y_train2)
probab_pca = clf_pca.predict_proba(X_test_pca2)
scores = roc_auc_score(y_test2,probab_pca[:,1])
scores

0.7574733584710601

In [207]:
clf_skb = clf.fit(X_train_skb2,y_train2)
probab_skb = clf_skb.predict_proba(X_test_skb2)
scores = roc_auc_score(y_test2,probab_skb[:,1])
scores

0.8264840335436348

In [243]:
ada_cut = AdaBoostClassifier(n_estimators = 8,random_state=23)
ada_cut.fit(X_train_cut1,y_train2)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=8, random_state=23)

In [244]:
sol_ada_cut = ada_cut.predict_proba(X_test_cut1)
sco_ada_cut = roc_auc_score(y_test2,sol_ada_cut[:,1])
sco_ada_cut

0.8264840335436348

#### Modelo Final

In [271]:
ada_final = AdaBoostClassifier(n_estimators = 8,random_state=23)
ada_final.fit(X_train_skb_compl,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=8, random_state=23)

In [ ]:
sol_ada_final = ada.predict_proba(X_test_skb_compl)
resultado_ada = pd.Series(sol_ada_final[:,1])
resultado_ada.index.name="USER_ID" # Renombramos las columnas
resultado_ada.name="SCORE" # Renombramos las columnas
resultado_ada.to_csv("respuesta_ada", header=True, compression="zip")

## RandomForest()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


param_grid = {
    "n_estimators" : [200,500,1000],
    "max_depth" : [3,4],
    "max_features" : [50,100,200],
}

random_forest = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini')

clasificador = GridSearchCV(random_forest,param_grid,scoring="roc_auc",cv=StratifiedKFold(5))

In [ ]:
forest2 = clasificador.fit(X_train2,y_train2)

In [ ]:
print(forest2.best_score_,forest2.best_estimator_)

In [ ]:
clf_for2 = forest2.best_estimator_

In [ ]:
sol_rf2 = clf_for2.predict_proba(X_test2)
scores = roc_auc_score(y_test2,sol_rf2[:,1])
scores

In [ ]:
rf_opt = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini',n_estimators=200,max_features=100,max_depth=4)
rf_opt_pca = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini',n_estimators=200,max_depth=4)
rf_opt_skb = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini',n_estimators=200,max_depth=4)

In [ ]:
rf_opt_pca.fit(X_train_pca2,y_train2)

In [ ]:
sol_rf22 = rf_opt_pca.predict_proba(X_test_pca2)
scores22 = roc_auc_score(y_test2,sol_rf22[:,1])
scores22

In [ ]:
rf_opt_skb.fit(X_train_skb2,y_train2)

In [ ]:
sol_rf_skb = rf_opt_skb.predict_proba(X_test_skb2)
scores = roc_auc_score(y_test2,sol_rf_skb[:,1])
scores

#### Modelo Final

In [269]:
from sklearn.ensemble import RandomForestClassifier

rf_final = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini',n_estimators=200,max_depth=4)

In [268]:
rf_final_sol = rf_final.predict_proba(X_test_skb_compl)
resultado_random = pd.Series(rf_final_sol[:,1])
resultado_random.index.name="USER_ID" # Renombramos las columnas
resultado_random.name="SCORE" # Renombramos las columnas
resultado_random.to_csv("respuesta_random", header=True, compression="zip")

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.


## CATBOOST

In [246]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
cat = CatBoostClassifier(random_seed=47)

grid_cat = {
    "iterations": [50,100,200],
    "max_depth": [None,2,4,6],
}

cat_clf = GridSearchCV(cat,grid_cat,cv=StratifiedKFold(5))

In [ ]:
cat_clf.fit(X_train2,y_train2)

In [ ]:
cat_clf.best_estimator_

In [ ]:
mod_cat2 = cat_clf.best_estimator_
sol_cat2 = mod_cat2.predict_proba(X_test2)
sco = roc_auc_score(y_test2, sol_cat2[:,1])
sco

In [247]:
from sklearn.metrics import roc_auc_score

mod_cat_nogs = CatBoostClassifier(iterations=220,random_seed=47,max_depth=None)
mod_cat_nogs.fit(X_train2,y_train2)
sol_cat_nogs = mod_cat_nogs.predict_proba(X_test2)
sco_cat_nogs = roc_auc_score(y_test2,sol_cat_nogs[:,1])
sco_cat_nogs

Learning rate set to 0.128292
0:	learn: 0.5166806	total: 228ms	remaining: 50s
1:	learn: 0.3982812	total: 295ms	remaining: 32.2s
2:	learn: 0.3060964	total: 358ms	remaining: 25.9s
3:	learn: 0.2562495	total: 421ms	remaining: 22.7s
4:	learn: 0.2229133	total: 498ms	remaining: 21.4s
5:	learn: 0.1899153	total: 564ms	remaining: 20.1s
6:	learn: 0.1701394	total: 632ms	remaining: 19.2s
7:	learn: 0.1596295	total: 695ms	remaining: 18.4s
8:	learn: 0.1522736	total: 768ms	remaining: 18s
9:	learn: 0.1444023	total: 829ms	remaining: 17.4s
10:	learn: 0.1388204	total: 890ms	remaining: 16.9s
11:	learn: 0.1352419	total: 952ms	remaining: 16.5s
12:	learn: 0.1322154	total: 1.03s	remaining: 16.4s
13:	learn: 0.1304865	total: 1.1s	remaining: 16.3s
14:	learn: 0.1288307	total: 1.17s	remaining: 16s
15:	learn: 0.1278227	total: 1.23s	remaining: 15.7s
16:	learn: 0.1258952	total: 1.3s	remaining: 15.5s
17:	learn: 0.1248161	total: 1.36s	remaining: 15.3s
18:	learn: 0.1235724	total: 1.42s	remaining: 15.1s
19:	learn: 0.123298

160:	learn: 0.0869445	total: 12.4s	remaining: 4.54s
161:	learn: 0.0867407	total: 12.5s	remaining: 4.46s
162:	learn: 0.0865109	total: 12.6s	remaining: 4.4s
163:	learn: 0.0863347	total: 12.7s	remaining: 4.33s
164:	learn: 0.0861019	total: 12.8s	remaining: 4.25s
165:	learn: 0.0858326	total: 12.8s	remaining: 4.18s
166:	learn: 0.0855781	total: 13s	remaining: 4.11s
167:	learn: 0.0853451	total: 13.1s	remaining: 4.04s
168:	learn: 0.0852068	total: 13.2s	remaining: 3.97s
169:	learn: 0.0849480	total: 13.2s	remaining: 3.9s
170:	learn: 0.0847537	total: 13.3s	remaining: 3.82s
171:	learn: 0.0845676	total: 13.4s	remaining: 3.75s
172:	learn: 0.0844851	total: 13.5s	remaining: 3.68s
173:	learn: 0.0842361	total: 13.6s	remaining: 3.6s
174:	learn: 0.0841120	total: 13.7s	remaining: 3.53s
175:	learn: 0.0839728	total: 13.8s	remaining: 3.45s
176:	learn: 0.0838052	total: 13.9s	remaining: 3.38s
177:	learn: 0.0836497	total: 14s	remaining: 3.31s
178:	learn: 0.0834419	total: 14.1s	remaining: 3.23s
179:	learn: 0.08317

0.8466555592990359

In [249]:
from sklearn.metrics import roc_auc_score

mod_cat_skb = CatBoostClassifier(iterations=220,random_seed=47,max_depth=None)
mod_cat_skb.fit(X_train_skb2,y_train2)
sol_cat_skb = mod_cat_skb.predict_proba(X_test_skb2)
sco_cat_skb = roc_auc_score(y_test2,sol_cat_skb[:,1])
sco_cat_skb

Learning rate set to 0.128292
0:	learn: 0.5148280	total: 21.7ms	remaining: 4.76s
1:	learn: 0.3941237	total: 40.9ms	remaining: 4.46s
2:	learn: 0.3002786	total: 60.7ms	remaining: 4.39s
3:	learn: 0.2508551	total: 80.4ms	remaining: 4.34s
4:	learn: 0.2174593	total: 100ms	remaining: 4.3s
5:	learn: 0.1930605	total: 120ms	remaining: 4.29s
6:	learn: 0.1741813	total: 141ms	remaining: 4.3s
7:	learn: 0.1606250	total: 161ms	remaining: 4.26s
8:	learn: 0.1504644	total: 182ms	remaining: 4.26s
9:	learn: 0.1417288	total: 205ms	remaining: 4.3s
10:	learn: 0.1365807	total: 235ms	remaining: 4.46s
11:	learn: 0.1327769	total: 257ms	remaining: 4.45s
12:	learn: 0.1292702	total: 281ms	remaining: 4.47s
13:	learn: 0.1264559	total: 302ms	remaining: 4.44s
14:	learn: 0.1249559	total: 326ms	remaining: 4.45s
15:	learn: 0.1230356	total: 346ms	remaining: 4.41s
16:	learn: 0.1221960	total: 366ms	remaining: 4.38s
17:	learn: 0.1210944	total: 389ms	remaining: 4.37s
18:	learn: 0.1203198	total: 413ms	remaining: 4.36s
19:	learn:

163:	learn: 0.0780193	total: 3.85s	remaining: 1.31s
164:	learn: 0.0776742	total: 3.88s	remaining: 1.29s
165:	learn: 0.0775203	total: 3.9s	remaining: 1.27s
166:	learn: 0.0774001	total: 3.92s	remaining: 1.24s
167:	learn: 0.0770061	total: 3.94s	remaining: 1.22s
168:	learn: 0.0769137	total: 3.96s	remaining: 1.19s
169:	learn: 0.0768658	total: 3.97s	remaining: 1.17s
170:	learn: 0.0767122	total: 3.99s	remaining: 1.14s
171:	learn: 0.0763488	total: 4.01s	remaining: 1.12s
172:	learn: 0.0762260	total: 4.03s	remaining: 1.09s
173:	learn: 0.0760846	total: 4.06s	remaining: 1.07s
174:	learn: 0.0758253	total: 4.08s	remaining: 1.05s
175:	learn: 0.0756602	total: 4.1s	remaining: 1.02s
176:	learn: 0.0754781	total: 4.12s	remaining: 1s
177:	learn: 0.0753493	total: 4.14s	remaining: 977ms
178:	learn: 0.0752534	total: 4.16s	remaining: 953ms
179:	learn: 0.0748838	total: 4.18s	remaining: 929ms
180:	learn: 0.0746851	total: 4.2s	remaining: 905ms
181:	learn: 0.0745294	total: 4.22s	remaining: 881ms
182:	learn: 0.0743

0.8502397622223351

In [251]:
otro_cat_skb = CatBoostClassifier(iterations=220,random_seed=47,max_depth=None)
otro_cat_skb.fit(X_train_cut1,y_train2)
sol_otro_cat = otro_cat_skb.predict_proba(X_test_cut1)
sco_otro_cat = roc_auc_score(y_test2,sol_otro_cat[:,1])
sco_otro_cat

Learning rate set to 0.128292
0:	learn: 0.5109799	total: 26.4ms	remaining: 5.79s
1:	learn: 0.3919170	total: 46.1ms	remaining: 5.03s
2:	learn: 0.3067675	total: 65.7ms	remaining: 4.75s
3:	learn: 0.2561366	total: 85.1ms	remaining: 4.59s
4:	learn: 0.2196749	total: 105ms	remaining: 4.51s
5:	learn: 0.1910263	total: 124ms	remaining: 4.44s
6:	learn: 0.1696219	total: 144ms	remaining: 4.39s
7:	learn: 0.1577037	total: 164ms	remaining: 4.35s
8:	learn: 0.1504543	total: 184ms	remaining: 4.31s
9:	learn: 0.1434988	total: 210ms	remaining: 4.41s
10:	learn: 0.1365936	total: 246ms	remaining: 4.67s
11:	learn: 0.1327284	total: 266ms	remaining: 4.61s
12:	learn: 0.1299587	total: 285ms	remaining: 4.54s
13:	learn: 0.1279901	total: 305ms	remaining: 4.49s
14:	learn: 0.1251700	total: 325ms	remaining: 4.45s
15:	learn: 0.1237480	total: 346ms	remaining: 4.41s
16:	learn: 0.1231162	total: 365ms	remaining: 4.36s
17:	learn: 0.1217756	total: 385ms	remaining: 4.32s
18:	learn: 0.1207655	total: 405ms	remaining: 4.28s
19:	lea

169:	learn: 0.0777087	total: 3.95s	remaining: 1.16s
170:	learn: 0.0773548	total: 3.98s	remaining: 1.14s
171:	learn: 0.0772068	total: 4s	remaining: 1.12s
172:	learn: 0.0771563	total: 4.03s	remaining: 1.09s
173:	learn: 0.0769937	total: 4.06s	remaining: 1.07s
174:	learn: 0.0768449	total: 4.09s	remaining: 1.05s
175:	learn: 0.0765924	total: 4.12s	remaining: 1.03s
176:	learn: 0.0764820	total: 4.14s	remaining: 1.01s
177:	learn: 0.0763029	total: 4.17s	remaining: 985ms
178:	learn: 0.0761393	total: 4.21s	remaining: 963ms
179:	learn: 0.0759043	total: 4.23s	remaining: 940ms
180:	learn: 0.0756223	total: 4.26s	remaining: 918ms
181:	learn: 0.0754531	total: 4.29s	remaining: 896ms
182:	learn: 0.0752614	total: 4.32s	remaining: 873ms
183:	learn: 0.0750174	total: 4.34s	remaining: 850ms
184:	learn: 0.0747677	total: 4.37s	remaining: 827ms
185:	learn: 0.0746006	total: 4.4s	remaining: 804ms
186:	learn: 0.0744963	total: 4.42s	remaining: 781ms
187:	learn: 0.0741107	total: 4.45s	remaining: 758ms
188:	learn: 0.07

0.8484769675963318

In [250]:
mod_cat_pca = mod_cat2.fit(X_train_pca2,y_train2)

NameError: name 'mod_cat2' is not defined

In [ ]:
sol_pca = mod_cat_pca.predict_proba(X_test2)
scores_sol_pca = scores(y_test2,sol_pca[:,1])
scores_sol_pca

#### Modelo Final

In [263]:
cat_final = CatBoostClassifier(iterations=220,random_seed=47,max_depth=None)
cat_final.fit(X_train_skb_compl,y_train)

Learning rate set to 0.128977
0:	learn: 0.5005976	total: 21.6ms	remaining: 4.72s
1:	learn: 0.3806351	total: 42.7ms	remaining: 4.65s
2:	learn: 0.2969616	total: 62.9ms	remaining: 4.55s
3:	learn: 0.2357071	total: 84.1ms	remaining: 4.54s
4:	learn: 0.2036473	total: 105ms	remaining: 4.52s
5:	learn: 0.1797855	total: 126ms	remaining: 4.5s
6:	learn: 0.1623803	total: 147ms	remaining: 4.47s
7:	learn: 0.1489238	total: 167ms	remaining: 4.44s
8:	learn: 0.1418975	total: 188ms	remaining: 4.42s
9:	learn: 0.1372203	total: 219ms	remaining: 4.59s
10:	learn: 0.1308658	total: 244ms	remaining: 4.64s
11:	learn: 0.1268436	total: 265ms	remaining: 4.59s
12:	learn: 0.1231206	total: 287ms	remaining: 4.57s
13:	learn: 0.1208905	total: 307ms	remaining: 4.52s
14:	learn: 0.1172973	total: 328ms	remaining: 4.49s
15:	learn: 0.1161940	total: 351ms	remaining: 4.47s
16:	learn: 0.1150113	total: 377ms	remaining: 4.5s
17:	learn: 0.1141059	total: 398ms	remaining: 4.46s
18:	learn: 0.1133132	total: 418ms	remaining: 4.43s
19:	learn

160:	learn: 0.0718889	total: 4.11s	remaining: 1.5s
161:	learn: 0.0717334	total: 4.14s	remaining: 1.48s
162:	learn: 0.0715999	total: 4.17s	remaining: 1.46s
163:	learn: 0.0713118	total: 4.2s	remaining: 1.43s
164:	learn: 0.0710984	total: 4.23s	remaining: 1.41s
165:	learn: 0.0710758	total: 4.25s	remaining: 1.38s
166:	learn: 0.0708673	total: 4.28s	remaining: 1.36s
167:	learn: 0.0704632	total: 4.31s	remaining: 1.33s
168:	learn: 0.0702190	total: 4.34s	remaining: 1.31s
169:	learn: 0.0700997	total: 4.37s	remaining: 1.29s
170:	learn: 0.0699151	total: 4.4s	remaining: 1.26s
171:	learn: 0.0696669	total: 4.43s	remaining: 1.24s
172:	learn: 0.0693873	total: 4.47s	remaining: 1.21s
173:	learn: 0.0690690	total: 4.51s	remaining: 1.19s
174:	learn: 0.0688919	total: 4.55s	remaining: 1.17s
175:	learn: 0.0687746	total: 4.58s	remaining: 1.15s
176:	learn: 0.0685853	total: 4.62s	remaining: 1.12s
177:	learn: 0.0682465	total: 4.65s	remaining: 1.1s
178:	learn: 0.0680030	total: 4.68s	remaining: 1.07s
179:	learn: 0.06

In [264]:
sol_cat = cat_final.predict_proba(X_test_skb_compl)
resultado_cat = pd.Series(sol_cat[:,1])
resultado_cat.index.name="USER_ID" # Renombramos las columnas
resultado_cat.name="SCORE" # Renombramos las columnas
resultado_cat.to_csv("respuesta_cat", header=True, compression="zip")

### XGBoosting

In [76]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(verbosity=2,max_depth=2,sub_sample=0.5)

In [77]:
xgb_clf.fit(X_train_skb2, y_train2)

[15:38:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

[15:38:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[15:38:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:38:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       sub_sample=0.5, subsample=1, verbosity=2)

In [78]:
sol_xgb = xgb_clf.predict_proba(X_test_skb2)
sco_xgb = roc_auc_score(y_test2,sol_xgb[:,1])
sco_xgb

0.8536379705823983

#### Modelo Final

In [79]:
xgb_final = XGBClassifier(verbosity=2,max_depth=2,sub_sample=0.5)
xgb_final.fit(X_train_skb_compl,y_train)

[15:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

[15:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[15:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[15:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pr

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       sub_sample=0.5, subsample=1, verbosity=2)

In [80]:
sol_xgb = xgb_final.predict_proba(X_test_skb_compl)
resultado_xgb = pd.Series(sol_xgb[:,1])
resultado_xgb.index.name="USER_ID" # Renombramos las columnas
resultado_xgb.name="SCORE" # Renombramos las columnas
resultado_xgb.to_csv("respuesta_xgb", header=True, compression="zip")

## Support Vector Machine

In [83]:
from sklearn.svm import SVC

svm = SVC(class_weight="balanced",probability=True,verbose=1,random_state=99,gamma="auto")

In [84]:
svm.fit(X_train_skb2,y_train2)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=99, shrinking=True,
  tol=0.001, verbose=1)

In [87]:
svm_prueba = svm.predict_proba(X_test_skb2)
scoring_svm = roc_auc_score(y_test2, svm_prueba[:,1])
scoring_svm

0.6048455506685189

## Clustering

In [89]:
from sklearn.cluster import KMeans 

kmeans = KMeans(n_clusters=2,random_state=44,verbose=1) 


## Red Neuronal

In [260]:
X_train_km = kmeans.fit_transform(X_train_skb2)
y_kmeans = kmeans.predict(X_test_skb2)

In [261]:
scoring = roc_auc_score(y_test2,y_kmeans)
scoring

0.49677070977925264

In [47]:
from keras.models import Sequential
from keras import layers
from sklearn.metrics import roc_auc_score
import tensorflow as tf

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

maxfeatures2 = X_train2.shape[1]
max_featuresskb = X_train_skb_compl.shape[1]

model = Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(max_featuresskb,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))
model.summary()
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy',auroc])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 16)                3216      
_________________________________________________________________
dense_26 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 17        
Total params: 3,505
Trainable params: 3,505
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.fit(X_train_skb_compl,y_train,batch_size=512,epochs=50,verbose=1,validation_split=0.2)

W0915 15:17:32.338497 139782887892800 deprecation.py:323] From /home/horacio/.conda/envs/ds/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0915 15:17:32.378849 139782887892800 deprecation_wrapper.py:119] From /home/horacio/.conda/envs/ds/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 9223 samples, validate on 2306 samples
Epoch 1/50


InvalidArgumentError: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0

# FINALES 

In [274]:
from lightgbm import LGBMClassifier

models = [
    xgb_final,
    cat_final,
    ada_final,
    rf_final,
    LGBMClassifier()
]

In [278]:
pip install vecstack

  Stored in directory: /home/horacio/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack
Note: you may need to restart the kernel to use updated packages.


In [282]:
from vecstack import stacking

S_train, S_test = stacking(models,                                             # list of models
                               X_train_skb_compl, y_train, X_test_skb_compl,   # data
                               regression=False,                               # classification task (if you need 
                                                                               #     regression - set to True)
                               mode='oof_pred',                                # mode: oof for train set, fit on full 
                                                                               #     train and predict test set once
                               needs_proba=True,                               # predict probabilities (if you need 
                                                                               #     class labels - set to False) 
                               save_dir='.',                                   # save result and log in current dir 
                                                                               #     (to disable saving - set to None)
                               n_folds=5,                                      # number of folds
                               stratified=True,                                # stratified split for folds
                               shuffle=True,                                   # shuffle the data
                               random_state=0,                                 # ensure reproducibility
                               verbose=1) 

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred]
n_models:     [5]

model  0:     [XGBClassifier]
[18:44:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots,

[18:44:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[18:44:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

[18:44:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

[18:44:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

[18:44:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[18:44:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[18:44:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:44:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[18:44:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

...

193:	learn: 0.0597679	total: 6.54s	remaining: 877ms
194:	learn: 0.0594680	total: 6.58s	remaining: 843ms
195:	learn: 0.0592645	total: 6.61s	remaining: 809ms
196:	learn: 0.0590365	total: 6.63s	remaining: 775ms
197:	learn: 0.0587541	total: 6.68s	remaining: 742ms
198:	learn: 0.0586345	total: 6.71s	remaining: 708ms
199:	learn: 0.0584405	total: 6.74s	remaining: 674ms
200:	learn: 0.0583182	total: 6.78s	remaining: 641ms
201:	learn: 0.0582399	total: 6.81s	remaining: 607ms
202:	learn: 0.0578695	total: 6.84s	remaining: 573ms
203:	learn: 0.0577882	total: 6.87s	remaining: 539ms
204:	learn: 0.0575360	total: 6.89s	remaining: 505ms
205:	learn: 0.0573841	total: 6.92s	remaining: 471ms
206:	learn: 0.0571892	total: 6.95s	remaining: 437ms
207:	learn: 0.0570511	total: 6.98s	remaining: 403ms
208:	learn: 0.0568898	total: 7.01s	remaining: 369ms
209:	learn: 0.0567176	total: 7.04s	remaining: 335ms
210:	learn: 0.0565918	total: 7.08s	remaining: 302ms
211:	learn: 0.0562615	total: 7.11s	remaining: 268ms
212:	learn: 

139:	learn: 0.0728836	total: 4.32s	remaining: 2.47s
140:	learn: 0.0726880	total: 4.35s	remaining: 2.44s
141:	learn: 0.0723498	total: 4.38s	remaining: 2.41s
142:	learn: 0.0721278	total: 4.41s	remaining: 2.38s
143:	learn: 0.0717861	total: 4.44s	remaining: 2.34s
144:	learn: 0.0715447	total: 4.47s	remaining: 2.31s
145:	learn: 0.0713387	total: 4.5s	remaining: 2.28s
146:	learn: 0.0712377	total: 4.53s	remaining: 2.25s
147:	learn: 0.0710466	total: 4.56s	remaining: 2.22s
148:	learn: 0.0707472	total: 4.59s	remaining: 2.19s
149:	learn: 0.0705187	total: 4.62s	remaining: 2.16s
150:	learn: 0.0703109	total: 4.65s	remaining: 2.12s
151:	learn: 0.0702231	total: 4.68s	remaining: 2.09s
152:	learn: 0.0698056	total: 4.71s	remaining: 2.06s
153:	learn: 0.0695755	total: 4.74s	remaining: 2.03s
154:	learn: 0.0692005	total: 4.78s	remaining: 2s
155:	learn: 0.0689444	total: 4.81s	remaining: 1.97s
156:	learn: 0.0686520	total: 4.84s	remaining: 1.94s
157:	learn: 0.0683874	total: 4.87s	remaining: 1.91s
158:	learn: 0.06

79:	learn: 0.0898046	total: 2.4s	remaining: 4.2s
80:	learn: 0.0893484	total: 2.44s	remaining: 4.19s
81:	learn: 0.0884286	total: 2.47s	remaining: 4.16s
82:	learn: 0.0882776	total: 2.5s	remaining: 4.13s
83:	learn: 0.0881953	total: 2.53s	remaining: 4.09s
84:	learn: 0.0879482	total: 2.56s	remaining: 4.06s
85:	learn: 0.0877387	total: 2.58s	remaining: 4.03s
86:	learn: 0.0874709	total: 2.62s	remaining: 4s
87:	learn: 0.0869909	total: 2.65s	remaining: 3.98s
88:	learn: 0.0868344	total: 2.68s	remaining: 3.95s
89:	learn: 0.0866541	total: 2.72s	remaining: 3.92s
90:	learn: 0.0859882	total: 2.75s	remaining: 3.9s
91:	learn: 0.0857431	total: 2.78s	remaining: 3.87s
92:	learn: 0.0855303	total: 2.81s	remaining: 3.84s
93:	learn: 0.0853880	total: 2.84s	remaining: 3.81s
94:	learn: 0.0853183	total: 2.88s	remaining: 3.78s
95:	learn: 0.0850109	total: 2.91s	remaining: 3.75s
96:	learn: 0.0847564	total: 2.94s	remaining: 3.72s
97:	learn: 0.0845291	total: 2.96s	remaining: 3.69s
98:	learn: 0.0840195	total: 3s	remaini

22:	learn: 0.1091881	total: 684ms	remaining: 5.86s
23:	learn: 0.1089212	total: 713ms	remaining: 5.83s
24:	learn: 0.1087109	total: 755ms	remaining: 5.89s
25:	learn: 0.1080783	total: 784ms	remaining: 5.85s
26:	learn: 0.1076914	total: 815ms	remaining: 5.82s
27:	learn: 0.1074091	total: 847ms	remaining: 5.8s
28:	learn: 0.1071551	total: 878ms	remaining: 5.78s
29:	learn: 0.1068412	total: 916ms	remaining: 5.8s
30:	learn: 0.1062757	total: 947ms	remaining: 5.77s
31:	learn: 0.1059905	total: 978ms	remaining: 5.74s
32:	learn: 0.1058023	total: 1.01s	remaining: 5.71s
33:	learn: 0.1053675	total: 1.05s	remaining: 5.73s
34:	learn: 0.1050188	total: 1.08s	remaining: 5.73s
35:	learn: 0.1044873	total: 1.13s	remaining: 5.75s
36:	learn: 0.1042297	total: 1.17s	remaining: 5.77s
37:	learn: 0.1040339	total: 1.2s	remaining: 5.74s
38:	learn: 0.1036754	total: 1.23s	remaining: 5.71s
39:	learn: 0.1034718	total: 1.26s	remaining: 5.67s
40:	learn: 0.1030740	total: 1.29s	remaining: 5.64s
41:	learn: 0.1026919	total: 1.32s	

186:	learn: 0.0664014	total: 6.51s	remaining: 1.15s
187:	learn: 0.0663214	total: 6.56s	remaining: 1.12s
188:	learn: 0.0660194	total: 6.6s	remaining: 1.08s
189:	learn: 0.0657185	total: 6.65s	remaining: 1.05s
190:	learn: 0.0655942	total: 6.69s	remaining: 1.01s
191:	learn: 0.0653670	total: 6.74s	remaining: 983ms
192:	learn: 0.0651664	total: 6.77s	remaining: 947ms
193:	learn: 0.0650982	total: 6.81s	remaining: 912ms
194:	learn: 0.0649512	total: 6.85s	remaining: 878ms
195:	learn: 0.0648553	total: 6.9s	remaining: 845ms
196:	learn: 0.0647275	total: 6.95s	remaining: 811ms
197:	learn: 0.0645681	total: 7s	remaining: 777ms
198:	learn: 0.0643555	total: 7.04s	remaining: 743ms
199:	learn: 0.0641633	total: 7.08s	remaining: 708ms
200:	learn: 0.0639667	total: 7.12s	remaining: 673ms
201:	learn: 0.0637174	total: 7.16s	remaining: 638ms
202:	learn: 0.0635222	total: 7.21s	remaining: 603ms
203:	learn: 0.0633235	total: 7.24s	remaining: 568ms
204:	learn: 0.0631398	total: 7.28s	remaining: 533ms
205:	learn: 0.062

In [283]:
model_of_models = xgb_final
model_of_models.fit(S_train,y_train)

[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[18:49:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pr

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       sub_sample=0.5, subsample=1, verbosity=2)

In [284]:
sol_final = model_of_models.predict_proba(S_test)
resultado_final = pd.Series(sol_final[:,1])
resultado_final.index.name="USER_ID" # Renombramos las columnas
resultado_final.name="SCORE" # Renombramos las columnas
resultado_final.to_csv("respuesta_final", header=True, compression="zip")